This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
!pip install -U chromadb pandas langchain langchain-openai langchain-chroma langchain-community

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import json
import pandas as pd
import chromadb
from io import StringIO
from langchain_openai import OpenAI
from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import SystemMessage, HumanMessage


os.environ["OPENAI_API_KEY"] = "REPLACE_WITH_YOUR_API_KEY"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"
client = chromadb.PersistentClient(path="./chroma_db") # Stores data in a local directory

llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=2000,
    api_key=os.getenv("OPENAI_API_KEY")
)

instruction = """
Generate 35 Real Estate lisitngs in CSV format with the following fields: Neighborhood, Price, Bedrooms, Bathrooms, House Size, Description.
Follow this format
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. 
This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels 
and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful 
hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious 
backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without 
compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to 
organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park
or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, 
commuting is a breeze.
"""

RE_listings = llm.invoke(instruction)
# Convert RE_listings->DataFrame
df = pd.read_csv(StringIO(RE_listings), sep=';')
df.to_csv('RE_listings.csv', index=False)

# Display All Listings
print("All HomeMatch Listings")
print(df)
print("--------HOMEMATCH--------")
print("\n")

# Load CSV
csv_loader = CSVLoader(file_path='./RE_listings.csv')
RE_listing_docs = csv_loader.load()

# Embeddings created
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",   
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")  
)

# Build Chroma Database
db = Chroma.from_documents(
    documents=RE_listing_docs,
    embedding=embeddings,
    collection_name="RE_listings",
    persist_directory="data",
)
#db.persist() not needed in current version; writes are automatically persisted 

All HomeMatch Listings
                                                  ```
0   Neighborhood,Price,Bedrooms,Bathrooms,House Si...
1   Green Oaks,$800,000,3,2,2,000 sqft,"Welcome to...
2   Sunnyvale,$600,000,4,3,2,500 sqft,"Discover th...
3   Maplewood,$450,000,2,1,1,200 sqft,"Charming 2-...
4   Riverdale,$750,000,3,2,1,800 sqft,"Welcome to ...
5   Cedar Park,$950,000,4,3,3,000 sqft,"This stunn...
6   Oakwood,$530,000,3,2,1,600 sqft,"Charming and ...
7   Hillside,$670,000,4,2,2,100 sqft,"Stunning vie...
8   Willow Creek,$590,000,2,2,1,400 sqft,"This bea...
9   Forest Glen,$430,000,3,1,1,800 sqft,"Welcome t...
10  Pine Hill,$725,000,5,4,3,500 sqft,"This exquis...
11  Lakeside,$480,000,3,2,1,400 sqft,"Charming 3-b...
12  Meadowbrook,$620,000,4,3,2,800 sqft,"Welcome t...
13  Crescent Heights,$710,000,3,2,2,200 sqft,"This...
14  Silver Valley,$540,000,2,2,1,500 sqft,"Charmin...
15  Brighton,$590,000,3,2,1,600 sqft,"Welcome to t...
16  Shady Pines,$400,000,2,1,1,000 sqft,"Cozy 2-be...
17  S

In [3]:
# Client Input
client_input = input("Welcome to House Match; Please enter what you are looking for: ")
print("You entered:", client_input)

# HomeMatch Search
results = db.similarity_search(client_input, k=5)
for i, d in enumerate(results, 1):
    print(f"\nHome Result {i}:\n{d.page_content}\nDetails: {d.metadata}") 

Welcome to House Match; Please enter what you are looking for: 3 bedroom
You entered: 3 bedroom

Home Result 1:
```: Brighton,$590,000,3,2,1,600 sqft,"Welcome to this beautifully updated 3-bedroom, 2-bathroom home in Brighton. Featuring a modern kitchen with stainless steel appliances and an open floor plan, this home is perfect for entertaining. The spacious master suite offers an en-suite bathroom for added privacy. Enjoy the spacious backyard with a deck. Brighton is known for its vibrant community and access to parks and trails."
Details: {'row': 15, 'source': './RE_listings.csv'}

Home Result 2:
---: Meadowview, $650,000, 4, 3, 2500 sqft, Discover your dream home in Meadowview! This stunning 4-bedroom, 3-bathroom residence offers a spacious layout and modern amenities. The gourmet kitchen is a chef's delight, featuring stainless steel appliances, granite countertops, and a large island. The expansive living room is perfect for entertaining, while the master suite comes with a luxu